In [1]:
import time
import keras

from keras.models import Model
from keras.layers import Dropout, Flatten, BatchNormalization, TimeDistributed, Input, Add, Concatenate,
from keras.layers import Dense, Conv2D, MaxPooling2D, LSTM, TimeDistributed, Reshape
import keras.backend as K
import keras.callbacks as callbacks

import pandas as pd
import numpy as np
from numpy import array
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [69]:

save_path = "/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/"
model_name = "3conv_only"
kf = KFold(n_splits = 10, shuffle =True)
num_epochs = 10

#### define functions ####

def model_cycle():
    inputs = Input(shape=(50, 4, 1))
        
    x = Conv2D(48, kernel_size=(3,4),
                   activation='relu',
                   padding='valid')(inputs)
    x = MaxPooling2D((2,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Conv2D(48, kernel_size=(11,1),
                activation='relu',
                padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Conv2D(48, kernel_size=(21,1),
                   activation='relu',
                   padding='same')(x)
    x = MaxPooling2D((2,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    print(f"pre reshape: {K.int_shape(x)}")

    # x = Reshape((1, 576))(x)
    x = Flatten()(x)

    print(f"post reshape: {K.int_shape(x)}")

    # x = Dense(20, activation='relu')(x)
    outputs = Dense(1, activation='linear')(x)

    # outputs = Dense(1, activation='linear')(x)
    print(outputs.shape)
    network = Model(inputs, outputs)
    network.compile(optimizer='rmsprop',
                    loss='mean_squared_error')
    return network
    
def dnaOneHot(sequence):
    seq_array = array(list(sequence))
    code = {"A": [0], "C": [1], "G": [2], "T": [3], "N": [4],
            "a": [0], "c": [1], "g": [2], "t": [3], "n": [4]}
    onehot_encoded_seq = []
    for char in seq_array:
        onehot_encoded = np.zeros(5)
        onehot_encoded[code[char]] = 1
        onehot_encoded_seq.append(onehot_encoded[0:4])
    return onehot_encoded_seq



In [61]:
def display_fits(fits):
    print(f"Average correlation on tiling: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 2])}",
          f"\nAverage MSE on tiling: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 2])}",
          f"\nAverage correlation on random: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 1])}",
          f"\nAverage MSE on random: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 1])}",
          f"\nAverage correlation on ChrV: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 3])}",
          f"\nAverage MSE on ChrV: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 3])}",
          f"\nAverage correlation on CN: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 0])}",
          f"\nAverage MSE on CN: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 0])}")

In [62]:
def find_c0new(dat):
  mat = np.empty((3,3), float)
  k = 2*np.pi/10.4
  n = array([26, 29, 31])
  mat[0:3,0] = 1
  mat[0:3, 1] = np.sin(n*k)
  mat[0:3, 2] = np.cos(n*k)
  inv_mat = np.linalg.inv(mat)
  c0A1A2 = array(np.matmul(dat[["n=26", "n=29", "n=31"]], np.transpose(inv_mat)))
  c0Aphi = c0A1A2
  c0Aphi[:,0] = c0A1A2[:,0]
  c0Aphi[:,1] = np.sqrt(c0A1A2[:,1]**2 + c0A1A2[:,2]**2)
  c0Aphi[:,2] <- np.sign(c0A1A2[:,2]) * np.arccos(c0A1A2[:,1]/c0Aphi[:,1])
  return c0Aphi[:,0]

In [49]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.process_time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.process_time() - self.epoch_time_start)
        
#### preparing data ####

data_cerevisiae_nucle = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle1.txt",delimiter = ",")
X1 = []
for sequence_nt in data_cerevisiae_nucle["Sequence"]:
    X1.append(dnaOneHot(sequence_nt))
X1 = array(X1)
X1 = X1.reshape((X1.shape[0],50,4,1))
X1_reverse = np.flip(X1,[1,2])
# Y1 = data_cerevisiae_nucle["C0"].values.astype(float)
Y1 = find_c0new(data_cerevisiae_nucle).astype(float)


In [50]:
data_random_library = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle3.txt",delimiter = ",")
X3 = []
for sequence_nt in data_random_library["Sequence"]:
    X3.append(dnaOneHot(sequence_nt))
X3 = array(X3)
X3 = X3.reshape((X3.shape[0],50,4,1))
X3_reverse = np.flip(X3,[1,2])
# Y3 = data_random_library["C0"].values.astype(float)
Y3 = find_c0new(data_random_library).astype(float)


In [51]:
data_tiling = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle5.txt",delimiter = ",")
X5 = []
for sequence_nt in data_tiling["Sequence"]:
    X5.append(dnaOneHot(sequence_nt))
X5 = array(X5)
X5 = X5.reshape((X5.shape[0],50,4,1))
X5_reverse = np.flip(X5,[1,2])
# Y5 = data_tiling["C0"].values.astype(float)
Y5 = find_c0new(data_tiling).astype(float)



In [52]:
data_chr5 = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle6.txt",delimiter = ",")
X6 = []
for sequence_nt in data_chr5["Sequence"]:
    X6.append(dnaOneHot(sequence_nt))
X6 = array(X6)
X6 = X6.reshape((X6.shape[0],50,4,1))
X6_reverse = np.flip(X6,[1,2])
# Y6 = data_chr5["C0"].values.astype(float)
Y6 = find_c0new(data_chr5).astype(float)

In [70]:
#### tiling

VALIDATION_LOSS = []
fold_var = 1
n = Y5.shape[0]

fits = []
detrend = []
times = []
times2 = []

for train_index, val_index in kf.split(Y5):
    training_X = X5[train_index]
    training_X_reverse = X5_reverse[train_index]
    validation_X = X5[val_index]
    validation_X_reverse = X5_reverse[val_index]
    training_Y = Y5[train_index]
    validation_Y = Y5[val_index]
    # CREATE NEW MODEL
    model = model_cycle()
    # CREATE CALLBACKS
    checkpoint = callbacks.ModelCheckpoint(save_path + model_name+"_tiling_"+str(fold_var)+".h5",
                                                    monitor='val_loss', verbose=1,
                                                    save_best_only=True, mode='min')
    time_callback = TimeHistory()

    history = model.fit(training_X, training_Y,
                        epochs=num_epochs,
                        callbacks= [checkpoint, time_callback],
                        validation_data=(validation_X, validation_Y))
    model.load_weights(save_path + model_name+"_tiling_"+str(fold_var)+".h5")
    model.save(save_path+model_name+"_tiling_"+str(fold_var),save_traces=False)
    times.append(time_callback.times)

    pred_Y = model.predict(training_X)
    pred_Y = pred_Y.reshape(pred_Y.shape[0])
    pred_Y_reverse = model.predict(training_X_reverse)
    pred_Y_reverse = pred_Y_reverse.reshape(pred_Y_reverse.shape[0])
    pred_Y = (pred_Y+pred_Y_reverse)/2
    reg =  LinearRegression().fit(array(pred_Y).reshape(-1, 1), array(training_Y).reshape(-1, 1))
    
    detrend_int = reg.intercept_
    detrend_slope = reg.coef_
    detrend.append([float(detrend_int), float(detrend_slope)])

    # start_time = time.process_time()
    # fit = model.predict(X1)
    # fit = fit.reshape(fit.shape[0])
    # fit_reverse = model.predict(X1_reverse)
    # fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    # reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    # fit = (fit + fit_reverse)/2
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit),reverse_corr]
    # fits.append(fit_tmp)
    # fit = detrend_int + fit * detrend_slope
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit),reverse_corr]
    # time0 = time.process_time() - start_time
    # times2.append([time0])
    # fits.append(fit_tmp)
    
    # start_time = time.process_time()
    # fit = model.predict(X3)
    # fit = fit.reshape(fit.shape[0])
    # fit_reverse = model.predict(X3_reverse)
    # fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    # reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    # fit = (fit + fit_reverse)/2
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit),reverse_corr]
    # fits.append(fit_tmp)
    # fit = detrend_int + fit * detrend_slope
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit),reverse_corr]
    # time0 = time.process_time() - start_time
    # times2.append([time0])
    # fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(validation_X)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(validation_X_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    # start_time = time.process_time()
    # fit = model.predict(X6)
    # fit = fit.reshape(fit.shape[0])
    # fit_reverse = model.predict(X6_reverse)
    # fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    # reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    # fit = (fit + fit_reverse)/2
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit),reverse_corr]
    # fits.append(fit_tmp)
    # fit = detrend_int + fit * detrend_slope
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit),reverse_corr]
    # time0 = time.process_time() - start_time
    # times2.append([time0])
    # fits.append(fit_tmp)
    
    K.clear_session()
    fold_var += 1
    
detrend = array(detrend)
detrend = pd.DataFrame(detrend)
detrend.to_csv(save_path +model_name+"_detrend_tiling.txt", index = False)

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+ "_fits_tiling.txt", index = False)

with open(save_path +model_name+"_time_tiling.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')

with open(save_path +model_name+"_pred_time_tiling.txt", "w") as file:
    for row in times2:
        s = " ".join(map(str, row))
        file.write(s+'\n')


pre reshape: (None, 12, 1, 48)
post reshape: (None, 576)
(None, 1)
Epoch 1/10
2315/2317 [============================>.] - ETA: 0s - loss: 0.1543
Epoch 1: val_loss improved from inf to 0.05536, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_1.h5
2317/2317 [==============================] - 28s 12ms/step - loss: 0.1543 - val_loss: 0.0554
Epoch 2/10
2313/2317 [============================>.] - ETA: 0s - loss: 0.0508
Epoch 2: val_loss improved from 0.05536 to 0.04262, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_1.h5
2317/2317 [==============================] - 32s 14ms/step - loss: 0.0508 - val_loss: 0.0426
Epoch 3/10
2314/2317 [============================>.] - ETA: 0s - loss: 0.0464
Epoch 3: val_loss improved from 0.04262 to 0.04226, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_1.h5
2317/2317 [=======================

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_1/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_1/assets


258/258 [==============================] - 2s 8ms/step
pre reshape: (None, 12, 1, 48)
post reshape: (None, 576)
(None, 1)
Epoch 1/10
2317/2317 [==============================] - ETA: 0s - loss: 0.1540
Epoch 1: val_loss improved from inf to 0.04996, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_2.h5
2317/2317 [==============================] - 48s 20ms/step - loss: 0.1540 - val_loss: 0.0500
Epoch 2/10
2316/2317 [============================>.] - ETA: 0s - loss: 0.0507
Epoch 2: val_loss improved from 0.04996 to 0.04168, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_2.h5
2317/2317 [==============================] - 44s 19ms/step - loss: 0.0507 - val_loss: 0.0417
Epoch 3/10
2315/2317 [============================>.] - ETA: 0s - loss: 0.0461
Epoch 3: val_loss improved from 0.04168 to 0.04097, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3c

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_2/assets


...

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_9/assets


258/258 [==============================] - 2s 9ms/step
pre reshape: (None, 12, 1, 48)
post reshape: (None, 576)
(None, 1)
Epoch 1/10
2315/2317 [============================>.] - ETA: 0s - loss: 0.1577
Epoch 1: val_loss improved from inf to 0.05359, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_10.h5
2317/2317 [==============================] - 53s 22ms/step - loss: 0.1577 - val_loss: 0.0536
Epoch 2/10
2316/2317 [============================>.] - ETA: 0s - loss: 0.0507
Epoch 2: val_loss improved from 0.05359 to 0.04335, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_10.h5
2317/2317 [==============================] - 53s 23ms/step - loss: 0.0507 - val_loss: 0.0433
Epoch 3/10
2317/2317 [==============================] - ETA: 0s - loss: 0.0463
Epoch 3: val_loss improved from 0.04335 to 0.04282, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_10/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/3conv_only_tiling_10/assets


258/258 [==============================] - 2s 7ms/step


In [74]:
# display_fits(fits)
print(f"Average correlation on tiling: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 2) == 0])}",
    f"\nAverage MSE on tiling: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 2) == 0])}")


Average correlation on tiling: 0.8714032667119962 
Average MSE on tiling: 0.038828389130763266
